In [8]:
import pandas as pd
import os

In [9]:
team_splits = pd.read_pickle(os.path.join('..', 'team_splits_periods.pickle'))
scoring = pd.read_pickle(os.path.join('..', 'scoring.pickle'))

In [10]:
ducks = team_splits.loc[team_splits["name"] == "Anaheim Ducks", :]
ducks = ducks.sort_index()
ducks.head(10)

,name,W,L,T,OL
month,,,,,
2006-01,Anaheim Ducks,4.0,5.0,NaN,2.0
2006-02,Anaheim Ducks,5.0,5.0,NaN,2.0
2006-03,Anaheim Ducks,10.0,3.0,NaN,2.0
2006-04,Anaheim Ducks,1.0,0.0,NaN,2.0
2006-10,Anaheim Ducks,9.0,0.0,NaN,3.0
2006-11,Anaheim Ducks,10.0,2.0,NaN,3.0
2006-12,Anaheim Ducks,9.0,5.0,NaN,0.0
2007-01,Anaheim Ducks,8.0,4.0,NaN,1.0
2007-02,Anaheim Ducks,10.0,2.0,NaN,1.0


In [13]:
#ducks.index = ducks.index.to_timestamp()
ducks.resample("YE").sum() #yearly summary for anaheim ducks

,name,W,L,T,OL
month,,,,,
2006-12-31,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,48.0,20.0,0.0,14.0
2007-12-31,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,47.0,27.0,0.0,8.0
2008-12-31,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,42.0,33.0,0.0,7.0
2009-12-31,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,39.0,32.0,0.0,11.0
2010-12-31,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,47.0,30.0,0.0,5.0
2011-12-31,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,34.0,36.0,0.0,12.0


In [17]:
ducks.index = ducks.index.map(
    lambda x: 
    x + pd.DateOffset(years=1) if x.month < 5
    else x
)
ducks = ducks.sort_index() #setting up the years
ducks.head()

,name,W,L,T,OL
month,,,,,
2006-10-01,Anaheim Ducks,9.0,0.0,NaN,3.0
2006-11-01,Anaheim Ducks,10.0,2.0,NaN,3.0
2006-12-01,Anaheim Ducks,9.0,5.0,NaN,0.0
2007-09-01,Anaheim Ducks,1.0,1.0,NaN,0.0
2007-10-01,Anaheim Ducks,3.0,6.0,NaN,2.0


In [18]:
resampler = ducks.resample("YE-JUN")
resampler

In [19]:
resampler.sum()

,name,W,L,T,OL
month,,,,,
2007-06-30,Anaheim DucksAnaheim DucksAnaheim Ducks,28.0,7.0,0.0,6.0
2008-06-30,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,39.0,30.0,0.0,13.0
2009-06-30,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,47.0,25.0,0.0,7.0
2010-06-30,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,39.0,35.0,0.0,10.0
2011-06-30,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,43.0,32.0,0.0,8.0
2012-06-30,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,37.0,34.0,0.0,7.0
2013-06-30,Anaheim DucksAnaheim DucksAnaheim DucksAnaheim...,24.0,15.0,0.0,6.0


In [20]:
# We can't to do this: resampler.expanding().sum()
res = resampler.apply(
        lambda x: x.reset_index()[["W", "L"]]
                   .expanding().sum()
)
res.head(8)

,W,L
0,9.0,0.0
1,19.0,2.0
2,28.0,7.0
0,1.0,1.0
1,4.0,7.0
2,12.0,11.0
3,19.0,17.0
4,23.0,22.0


In [21]:
res.index = ducks.index
res.head()

,W,L
month,,
2006-10-01,9.0,0.0
2006-11-01,19.0,2.0
2006-12-01,28.0,7.0
2007-09-01,1.0,1.0
2007-10-01,4.0,7.0


In [22]:
final = res.asfreq(pd.tseries.offsets.MonthBegin())
final.head(10)

,W,L
month,,
2006-10-01,9.0,0.0
2006-11-01,19.0,2.0
2006-12-01,28.0,7.0
2007-01-01,NaN,NaN
2007-02-01,NaN,NaN
2007-03-01,NaN,NaN
2007-04-01,NaN,NaN
2007-05-01,NaN,NaN
2007-06-01,NaN,NaN


In [23]:
fdf = final.reset_index()
fdf.head()

,month,W,L
0,2006-10-01,9.0,0.0
1,2006-11-01,19.0,2.0
2,2006-12-01,28.0,7.0
3,2007-01-01,NaN,NaN
4,2007-02-01,NaN,NaN


In [24]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [25]:
from bokeh.models.sources import ColumnDataSource as cds

In [26]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green")
p.line(source=source, x='month', y="L", color="red");

In [27]:
show(p)

In [30]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green", legend_label="Wins")
p.line(source=source, x='month', y="L", color="red", legend_label="Loses")
p.legend.click_policy="hide";

In [31]:
show(p)

In [32]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green", legend_label="Wins")
p.line(source=source, x='month', y="L", color="red", legend_label="Loses")
p.legend.click_policy="hide"
p.title.text = "Anaheim Ducks Performance by Season"
p.title.text_font_size = "25px"
p.title.align = "center";

In [33]:
show(p)

In [34]:
wayne = scoring.set_index("playerID").loc['gretzwa01']
wayne.head()

,firstName,lastName,pos,year,tmID,GP,G,A,Pts,SOG
playerID,,,,,,,,,,
gretzwa01,Wayne,Gretzky,C,1980,EDM,80.0,55.0,109.0,164.0,261.0
gretzwa01,Wayne,Gretzky,C,1981,EDM,80.0,92.0,120.0,212.0,369.0
gretzwa01,Wayne,Gretzky,C,1982,EDM,80.0,71.0,125.0,196.0,348.0
gretzwa01,Wayne,Gretzky,C,1983,EDM,74.0,87.0,118.0,205.0,324.0
gretzwa01,Wayne,Gretzky,C,1984,EDM,80.0,73.0,135.0,208.0,358.0


In [35]:
wayne = wayne[['year', 'tmID', 'GP', 'Pts']]
wayne = wayne.assign(pts_per_game = lambda x: x['Pts'] / x['GP'])
wayne.head()

,year,tmID,GP,Pts,pts_per_game
playerID,,,,,
gretzwa01,1980,EDM,80.0,164.0,2.05000
gretzwa01,1981,EDM,80.0,212.0,2.65000
gretzwa01,1982,EDM,80.0,196.0,2.45000
gretzwa01,1983,EDM,74.0,205.0,2.77027
gretzwa01,1984,EDM,80.0,208.0,2.60000


In [36]:
wayne.tmID.head()

playerID
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
Name: tmID, dtype: object

In [38]:
wayne['tmID'] = wayne['tmID'].astype('category')
wayne.loc[:, "tmID"] = wayne.tmID.cat.remove_unused_categories()
wayne.tmID.head()

playerID
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
Name: tmID, dtype: category
Categories (4, object): ['EDM', 'LAK', 'NYR', 'STL']

In [39]:
wayne.loc[:, "tmCode"] = wayne["tmID"].cat.codes
wayne.sample(5)

,year,tmID,GP,Pts,pts_per_game,tmCode
playerID,,,,,,
gretzwa01,1990,LAK,78.0,163.0,2.089744,1
gretzwa01,1983,EDM,74.0,205.0,2.770270,0
gretzwa01,1985,EDM,80.0,215.0,2.687500,0
gretzwa01,1997,NYR,82.0,90.0,1.097561,2
gretzwa01,1998,NYR,70.0,62.0,0.885714,2


In [40]:
wayne.loc[:, "year"] = pd.to_datetime(wayne.year, format="%Y")

/var/folders/_9/zr872_21565csjpn6gc7hdp80000gn/T/ipykernel_85023/2268399343.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['1980-01-01 00:00:00', '1981-01-01 00:00:00', '1982-01-01 00:00:00',
 '1983-01-01 00:00:00', '1984-01-01 00:00:00', '1985-01-01 00:00:00',
 '1986-01-01 00:00:00', '1987-01-01 00:00:00', '1988-01-01 00:00:00',
 '1989-01-01 00:00:00', '1990-01-01 00:00:00', '1991-01-01 00:00:00',
 '1992-01-01 00:00:00', '1993-01-01 00:00:00', '1994-01-01 00:00:00',
 '1995-01-01 00:00:00', '1995-01-01 00:00:00', '1996-01-01 00:00:00',
 '1997-01-01 00:00:00', '1998-01-01 00:00:00']
Length: 20, dtype: datetime64[ns]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  wayne.loc[:, "year"] = pd.to_datetime(wayne.year, format="%Y")


In [41]:
gp_max = wayne.GP.max()
gp_min = wayne.GP.min()
pts_per_game_max = wayne.pts_per_game.max()
pts_per_game_min = wayne.pts_per_game.min()

In [42]:
wayne.loc[:, "height"] = wayne.GP / gp_max
wayne.loc[:, "bottom"] = wayne.tmCode
wayne.loc[:, "top"] = wayne.bottom + wayne.height
wayne.head()

,year,tmID,GP,Pts,pts_per_game,tmCode,height,bottom,top
playerID,,,,,,,,,
gretzwa01,1980-01-01,EDM,80.0,164.0,2.05000,0,0.975610,0,0.975610
gretzwa01,1981-01-01,EDM,80.0,212.0,2.65000,0,0.975610,0,0.975610
gretzwa01,1982-01-01,EDM,80.0,196.0,2.45000,0,0.975610,0,0.975610
gretzwa01,1983-01-01,EDM,74.0,205.0,2.77027,0,0.902439,0,0.902439
gretzwa01,1984-01-01,EDM,80.0,208.0,2.60000,0,0.975610,0,0.975610


In [43]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=1, source=src);

In [44]:
show(p)

In [45]:
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=1, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [46]:
show(p)

In [47]:
reordered = wayne.sort_values('year').tmID.unique()
reordered

['EDM', 'LAK', 'STL', 'NYR']
Categories (4, object): ['EDM', 'LAK', 'NYR', 'STL']

In [48]:
wayne.loc[:, "tmID"] = wayne.tmID.cat.reorder_categories(reordered)
wayne.loc[:, "tmCode"] = wayne["tmID"].cat.codes
wayne.loc[:, "bottom"] = wayne.tmCode
wayne.loc[:, "top"] = wayne.bottom + wayne.height

In [49]:
width = 320 * 24 * 60 * 60 * 1000

In [50]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=width, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [51]:
show(p)

In [52]:
wayne.loc[:, "bottom"] = wayne.tmCode - (wayne.height / 2.0 ) + 0.5
wayne.loc[:, "top"] = wayne.bottom + wayne.height

/var/folders/_9/zr872_21565csjpn6gc7hdp80000gn/T/ipykernel_85023/531593554.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.01219512 0.01219512 0.01219512 0.04878049 0.01219512 0.01219512
 0.01829268 0.1097561  1.02439024 1.05487805 1.02439024 1.04878049
 1.22560976 1.00609756 1.20731707 1.12195122 3.3902439  2.
 2.         2.07317073]' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  wayne.loc[:, "bottom"] = wayne.tmCode - (wayne.height / 2.0 ) + 0.5


In [53]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=width, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [54]:
show(p)

In [55]:
wayne.loc[:, "height"] = wayne.GP / gp_max * 0.8
wayne.loc[:, "bottom"] = wayne.tmCode - (wayne.height / 2.0 ) + 0.5
wayne.loc[:, "top"] = wayne.bottom + wayne.height

In [56]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=width, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [57]:
show(p)

In [58]:
from bokeh.models import LinearColorMapper
color_mapper = LinearColorMapper(
    palette="Plasma256", 
    low=pts_per_game_min, 
    high=pts_per_game_max)

In [59]:
src = cds(wayne)
p = figure(x_axis_type='datetime', 
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom', top='top',
       x='year', width=width,
       color={'field': 'pts_per_game', 'transform': color_mapper},
       source=src);

In [60]:
show(p)

In [61]:
from bokeh.models import ColorBar
src = cds(wayne)
p = figure(x_axis_type='datetime', 
           width=500,
           height=500, 
           y_range=list(wayne.tmID.cat.categories)
)
p.vbar(bottom='bottom', top='top', x='year', width=width,
       color={'field': 'pts_per_game', 'transform': color_mapper}, source=src)
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8, border_line_color=None, 
                     location=(0,0)
)

p.add_layout(color_bar, 'right');

In [62]:
show(p)

In [63]:
from bokeh.models.tools import HoverTool
hover = HoverTool(
    tooltips=[
        ("Points per game", "@pts_per_game"),
        ("Games played", "@GP")
    ]
)

In [64]:
src = cds(wayne)
p = figure(x_axis_type='datetime', 
           width=500,
           height=500, 
           y_range=list(wayne.tmID.cat.categories)
)
p.vbar(bottom='bottom', top='top', x='year', width=width,
       color={'field': 'pts_per_game', 'transform': color_mapper}, source=src)
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8, border_line_color=None, 
                     location=(0,0)
)

p.add_layout(color_bar, 'right')
p.add_tools(hover);

In [65]:
show(p)